In [1]:
import pandas as pd
import numpy as np
import os.path

from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

from flyyufelix_models.resnet_101 import resnet101_model
from flyyufelix_models.custom_layers.scale_layer import Scale

import threading
lock = threading.Lock()

from keras.callbacks import ModelCheckpoint

from BsonIterator import BSONIterator

Using TensorFlow backend.


In [2]:
METADATA_DIR = 'metadata'
DATA_DIR = '../../dataset'

train_offsets_df = pd.read_csv(os.path.join(METADATA_DIR,"train_offsets.csv"), index_col=0)
train_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Train_0.0_0.05.csv"), index_col=0)
val_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Val_0.0_0.05.csv"), index_col=0)

num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 32

# Create a generator for training and a generator for validation.
# Tip: use ImageDataGenerator for data augmentation and preprocessing.
assert os.path.exists(os.path.join(DATA_DIR, 'train.bson'))
train_bson_file = open(os.path.join(DATA_DIR, 'train.bson'), "rb")
train_datagen = ImageDataGenerator()
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df,
                         num_classes, train_datagen, lock, target_size = (224,224),
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock, target_size = (224,224),
                       batch_size=batch_size, shuffle=True)

/home/ankdesh/virtualenvs/cdiscount-py2.7/local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Found 11757587 images belonging to 5270 classes.
Found 613706 images belonging to 5270 classes.


In [3]:
from keras.callbacks import ModelCheckpoint

# Save model Dir
MODEL_DIR = 'saved_models'
IMAGE_ROWS = 224
IMAGE_COLS = 224
CHANNELS =3

# Checkpointer 
checkpointer = ModelCheckpoint(filepath='saved_models/Resnet101.{epoch:02d}-{acc:.2f}-{val_acc:.2f}_RandomSplit_Train_0.5_0.1.h5', verbose=1)

In [4]:
model = load_model('saved_models/Resnet101.01-0.63-0.65_RandomSplit_Train_0.5_0.1.h5',custom_objects={
                   'Scale':Scale})

/home/ankdesh/virtualenvs/cdiscount-py2.7/local/lib/python2.7/site-packages/keras/engine/topology.py:1252: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [5]:
for layer in model.layers:
    layer.trainable = True

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
from keras.optimizers import SGD
opt = SGD(lr=1e-4, decay=0.0, momentum=0.9, nesterov=True)
model.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics=["accuracy"])


In [7]:
num_epochs = 2
batch_size = 16

# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4, callbacks=[checkpointer])


Epoch 1/2


ResourceExhaustedError: OOM when allocating tensor with shape[32,16,16,256]
	 [[Node: res4b15_branch2b_zeropadding/Pad = Pad[T=DT_FLOAT, Tpaddings=DT_INT32, _device="/job:localhost/replica:0/task:0/gpu:0"](res4b15_branch2a_relu/Relu, res4b15_branch2b_zeropadding/Pad/paddings)]]
	 [[Node: metrics_1/acc/Mean/_8091 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_59243_metrics_1/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'res4b15_branch2b_zeropadding/Pad', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ankdesh/virtualenvs/cdiscount-py2.7/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launc